In [ ]:
import time
import numpy as np

import torch, torchvision
import torch.nn as nn
import torch.optim as optim 
from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pack_padded_sequence  # padding our sequences


from DatasetClasses import TweetEmbeddingDataset
from LSTM_Models import LSTM_EMB_T    # LSTM (input embeddings, time scalars) -> list[0,1,]



In [ ]:
sen_embedding_path = "/home/public/tweetdatanlp/sent-trans-dbs/tweet_embeddings.npy"
meta_path = "/home/public/tweetdatanlp/sent-trans-dbs/tweet_metadata.csv"

# init dataset
data=TweetEmbeddingDataset(sen_embedding_path, meta_path, distance_threshold=0)

total_len = len(data)
train_pct, val_pct, test_pct = 0.8, 0.1, 0.1
train_len = int(train_pct * total_len)
val_len = int(val_pct * total_len)
test_len = total_len - train_len - val_len


train_set, val_set, test_set = random_split(data, [train_len, val_len, test_len])

# custom collate function for handling sequence padding (packed_pad_seq or something)
def collate_fn(batch):
    pass

batch_size = 1  # can increase if GPU memory allows (sequences aready big (2-64), 384) shape

# Train, test, validation
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

In [ ]:


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = LSTM_EMB_T(
    sen_embedding_size=384,  # Embeddings are of size  384s
    lstm_hidden_size=64,     # can customize
    time_embedding_size=64,  # can customize
    lstm_layer=1
).to(device)

criterion = nn.BCELoss()                # Binary cross entropy for 0/1 target
optimizer = optim.Adam(model.parameters(), lr=1e-3)

#### Training Loop